In [1]:
%env CUDA_VISIBLE_DEVICES=

env: CUDA_VISIBLE_DEVICES=


In [2]:
from magma.magma import Magma 
import torch 
import seaborn as sns
import matplotlib.pyplot as plt
import os 
from activations.torch import Rational
import numpy as np
from magma.utils import get_tokenizer
from magma.datasets import ImgCptDataset
from magma.transforms import get_transforms
from magma.config import MultimodalConfig
config = MultimodalConfig.from_yml(
            './../fb20-dgx2-configs/dev.yml'
        )
sns.set_style("darkgrid")
sns.color_palette('husl', 9)
from copy import deepcopy


In [4]:
tokenizer = get_tokenizer()
transforms = get_transforms(0, 'clip', 384)

new_250 = Magma.from_checkpoint(checkpoint_path='/home/ml-mmeuer/adaptable_magma/model_checkpoints/2_250.pt', config_path='./../fb20-dgx2-configs/dev.yml')    
# new_500 = Magma.from_checkpoint(checkpoint_path='/home/ml-mmeuer/adaptable_magma/model_checkpoints/2_500.pt', config_path='./../fb20-dgx2-configs/dev.yml').cuda()       
dataset = ImgCptDataset(config.eval_dataset_dir, new_250.tokenizer, new_250.transforms)
# model_7250 = Magma(config='./../fb20-dgx2-configs/dev.yml').cuda()

Loading GPTJ language model...
From EleutherAI/gpt-j-6B
loading magma checkpoint from: /home/ml-mmeuer/adaptable_magma/model_checkpoints/2_250.pt
magma successfully loaded


In [20]:
for n,m in new_250.named_parameters(): 
    print(n, m.requires_grad)

lm.transformer.wte.weight False
lm.transformer.h.0.ln_1.weight False
lm.transformer.h.0.ln_1.bias False
lm.transformer.h.0.attn.k_proj.weight False
lm.transformer.h.0.attn.v_proj.weight False
lm.transformer.h.0.attn.q_proj.weight False
lm.transformer.h.0.attn.out_proj.weight False
lm.transformer.h.0.mlp.0.fc_in.weight False
lm.transformer.h.0.mlp.0.fc_in.bias False
lm.transformer.h.0.mlp.0.fc_out.weight False
lm.transformer.h.0.mlp.0.fc_out.bias False
lm.transformer.h.0.mlp.1.switch_logits True
lm.transformer.h.0.mlp.1.adapter.0.weight True
lm.transformer.h.0.mlp.1.adapter.0.bias True
lm.transformer.h.0.mlp.1.adapter.1.f.numerator True
lm.transformer.h.0.mlp.1.adapter.1.f.denominator True
lm.transformer.h.0.mlp.1.adapter.2.weight True
lm.transformer.h.0.mlp.1.adapter.2.bias True
lm.transformer.h.1.ln_1.weight False
lm.transformer.h.1.ln_1.bias False
lm.transformer.h.1.attn.k_proj.weight False
lm.transformer.h.1.attn.v_proj.weight False
lm.transformer.h.1.attn.q_proj.weight False
lm.tra

In [18]:
def input_dist_forwards(model, dataset, num_samples=10):
    def grad_hook(module, input, output):
        module.input.append( input[0].detach().cpu().tolist())
        
    for i,(n,m) in enumerate(model.named_modules()):
        if isinstance(m, Rational):
            setattr(m,'input', [])
            m.register_forward_hook(grad_hook)
            
    for i in range(num_samples):
        images , captions = dataset[i]
        images, captions = images, captions
        model.forward(images, captions)


In [6]:
def return_dynamic_fig(model,cols=2 ,title='Dynamic Softmax', switch_temp=1.0,type=Rational):
    i=0
    for n,p in model.named_modules():
        if isinstance(p, type):
            i += cols
        rows = i // cols + 1
        rest = i % cols
        if rest != 0:
            rows += 1  
            
    Position = range(1,i + 1)

    fig = plt.figure(1, figsize=(20,80))

    fig.suptitle(title, fontsize=20, )
    return fig, Position, rows, cols

In [13]:
def plot_input_dist(model, cols=2, title='Distribution over input of rational activations'):
    fig, Position, rows, cols = return_dynamic_fig(model,cols=2, title="Title")
    j = 0
    for n,m in model.named_modules():
        if isinstance(m, Rational):
            linsp = torch.linspace(-3,3,100)
            y = m.forward(linsp)

            ax1 = fig.add_subplot(rows,cols,Position[j])
            ax2 = ax1.twinx()
            input_data = m.input.detach().numpy()
            input_filter_range = np.where((input_data > -3) & (input_data < 3))
            sns.distplot(m.input, ax=ax1, color='red')
            sns.lineplot(x=linsp.numpy(), y=y.numpy(), ax=ax2, color='blue')
            j+= 1
    fig.tight_layout()
    fig.subplots_adjust(top=0.95)
    plt.show()

In [19]:
input_dist_forwards(new_250, dataset, num_samples=10)
plot_input_dist(new_250, 2, title='Distribution over input of rational activations')

RuntimeError: "slow_conv2d_cpu" not implemented for 'Half'

In [ ]:
fig, Position, rows, cols = return_dynamic_fig(model_7250,cols=1, title="Distribution over input and output of rational activations")
palette = sns.color_palette('husl', 9)
j=0
for i,(n,m) in enumerate(model_7250.named_modules()):
    if isinstance(m, Rational):
        print(j)
        ax1 = fig.add_subplot(rows,cols,Position[j])
        ax2 = ax1.twinx()
        m_np = np.array(deepcopy(m.input))
        print(m_np.shape)
        sns.histplot(m_np.flatten(), ax=ax1)
        ax1.set_xlim(-3,3)
        ax2.set_xlim(-3,3)
        linsp = torch.linspace(-3,3,100).cuda()
        
        y = m.forward(linsp)
        
        sns.lineplot(x=linsp.detach().cpu().numpy(),y=y.detach().cpu().numpy(), ax=ax2, color=palette[1])
        ax2.grid(False)
        ax1.set_title(f'Input Distribution at Layer {j}')
        j+=1
        print('------------------------')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()

0
------------------------
1
------------------------
2
------------------------
3
------------------------
4
------------------------
5
------------------------
6
------------------------
7
------------------------
8
------------------------
9
------------------------
10
------------------------
11
------------------------
12
------------------------
13
------------------------
14
------------------------
15
------------------------
16
------------------------
17
------------------------
18
------------------------
19
------------------------
20
------------------------
21
------------------------
22
------------------------
23
------------------------
24
------------------------
25
------------------------
26
------------------------
27
------------------------
28
------------------------
29
------------------------
30
------------------------
31
------------------------
32
------------------------
33
------------------------
34
------------------------
35
------------------------
36

<Figure size 2000x8000 with 0 Axes>

In [ ]:
g = torch.distributions.Gumbel(0,1)
switch_temp = 0.1

# Create a matplot lib figure with subplots
i = 0


j = 0
for n,p in model_7250.named_parameters():
    if 'switch' in n: 
        res = []
        for i in range(1000):
            g_sample= g.sample(p.shape).to('cpu')
            sim = torch.softmax((p + g_sample)/switch_temp, dim=-1)
            res.append(sim)
        
        y_labels = ['switch_0', 'switch_1']
        res = torch.stack(res).detach().cpu().numpy()
        ax = fig.add_subplot(rows,cols,Position[j])
        ax.hist(res[:,0],  bins=25, label='switch_adapter')
        ax.set_title(f'Distribution of Adapter Weights at Layer {j//2}')
        ax = fig.add_subplot(rows,cols,Position[(j+1)])
        ax.set_title(f'Distribution of Identity Weights at Layer {j//2}')
        ax.hist(res[:,1],  bins=25,label='switch_identity')
        j += 2
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()
        

IndexError: range object index out of range